In [1]:
%%time
import numpy as np 
import keras 
import keras 
from keras.preprocessing.image import img_to_array 
from sklearn.preprocessing import LabelBinarizer 
from sklearn.model_selection import train_test_split 
from keras.models import Sequential, load_model 
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D 
from keras.callbacks import ModelCheckpoint 
from imutils import paths 
import random 
import pickle 
import cv2 
import os

BatchSize = 128
Classes = 4 
Epochs = 20 
InputShape = (64, 64, 3) 
data = [] 
labels = []

print("Loading training data...") 
imagePaths = sorted(list(paths.list_images('C:/Training/Capstone/train/'))) 
random.seed(42) 
random.shuffle(imagePaths) 

# loop over the input images 
for imagePath in imagePaths: 
    image = cv2.imread(imagePath, cv2.COLOR_RGB2GRAY) 
    image = img_to_array(image) 
    data.append(image) 
    label = imagePath.split(os.path.sep)[-2].split("_")[0] 
    # train images are spread across four folders based on their classes 
    if 'ZERO' in label: 
        labels.append('ZERO') 
    if 'ONE' in label: 
        labels.append('ONE') 
    if 'TWO' in label: 
        labels.append('TWO') 
    if 'THREE' in label: 
        labels.append('THREE') 

# scale the raw pixel intensities to the range [0, 1] 
data = np.array(data, dtype="float") / 255.0 
labels = np.array(labels) 

# binarize the labels 
mlb = LabelBinarizer() 
labels = mlb.fit_transform(labels) 

# partition the data into training and test splits (85/15) 
(x_train, x_test, y_train, y_test) = train_test_split(data, labels, test_size=0.15, random_state=42) 

# construct our model 
model = Sequential() 
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=InputShape)) 
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25)) 
model.add(Flatten()) 
model.add(Dense(128, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(Classes, activation='softmax')) 

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(), 
              metrics=['accuracy']) 

filepath = "model.h5" 
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min') 
callbacks_list = [checkpoint] 

# start training 
model.fit(x_train, y_train, batch_size=BatchSize, epochs=Epochs, verbose=1, 
          validation_data=(x_test, y_test), callbacks=callbacks_list) 
score = model.evaluate(x_test, y_test, verbose=0) 
print('Test loss:', score[0]) 
print('Test accuracy:', score[1]) 



C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Loading training data...
Train on 34000 samples, validate on 6000 samples
Epoch 1/20
34000/34000 [==============================] - 36s 1ms/step - loss: 0.4312 - acc: 0.8414 - val_loss: 0.0223 - val_acc: 0.9960

Epoch 00001: loss improved from inf to 0.43120, saving model to model.h5
Epoch 2/20
34000/34000 [==============================] - 33s 973us/step - loss: 0.0231 - acc: 0.9934 - val_loss: 0.0062 - val_acc: 0.9992

Epoch 00002: loss improved from 0.43120 to 0.02311, saving model to model.h5
Epoch 3/20
34000/34000 [==============================] - 33s 975us/step - loss: 0.0104 - acc: 0.9971 - val_loss: 0.0072 - val_acc: 0.9982

Epoch 00003: loss improved from 0.02311 to 0.01039, saving model to model.h5
Epoch 4/20
34000/34000 [==============================] - 33s 975us/step - loss: 0.0065 - acc: 0.9981 - val_loss: 0.0068 - val_acc: 0.9985

Epoch 00004: loss improved from 0.01039 to 0.00649, saving model to model.h5
Epoch 5/20
34000/34000 [==============================] - 33s 97

In [2]:
%%time
# save the model and multi-label binarizer to disk so we can classify our submission later 
model.save('capstone.model') 
f = open('capstone.pickle', "wb") 
f.write(pickle.dumps(mlb)) 
f.close() 

Wall time: 411 ms
